<a href="https://colab.research.google.com/github/paolaCaushi/anomalyDetectionWithTransferLearning/blob/main/clustering1249Rows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
from sklearn.cluster import KMeans
import folium

# Load the Excel file
file_path = '1249RowsTotal.xlsx'
data = pd.read_excel(file_path)

# Extract latitude and longitude columns
geo_data = data[['Latitudine', 'Longitudine']].dropna()

# Function to calculate geographical distance
def calculate_distance(p1, p2):
    return distance(p1, p2).meters  # Convert distance to meters

# Custom distance function for K-Means clustering
def custom_distance(X, Y):
    n_samples_X, n_samples_Y = X.shape[0], Y.shape[0]
    D = np.empty((n_samples_X, n_samples_Y))
    for i in range(n_samples_X):
        for j in range(n_samples_Y):
            D[i, j] = calculate_distance(X[i], Y[j])
    return D

# Number of clusters
num_clusters = 3

# Apply K-Means clustering with custom distance metric
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(geo_data)

# Add cluster labels to original data
geo_data['cluster'] = kmeans.labels_

# Convert cluster labels to integers
geo_data['cluster'] = geo_data['cluster'].astype(int)

# Get cluster centroids
centroids = kmeans.cluster_centers_

# Create a map centered at the average location
m = folium.Map(location=[geo_data['Latitudine'].mean(), geo_data['Longitudine'].mean()], zoom_start=10)

# Define cluster colors
cluster_colors = ['red', 'blue', 'green', 'purple', 'orange']

# Add points to the map
for _, row in geo_data.iterrows():
    folium.CircleMarker(location=[row['Latitudine'], row['Longitudine']],
                        radius=5,
                        color=cluster_colors[int(row['cluster'])],  # Ensure row['cluster'] is cast to int
                        fill=True).add_to(m)

# Add centroids to the map
for i, centroid in enumerate(centroids):
    folium.Marker(location=centroid,
                  popup=f'Cluster {i} centroid\nLat: {centroid[0]:.6f}, Lon: {centroid[1]:.6f}',
                  icon=folium.Icon(color=cluster_colors[i], icon='info-sign')).add_to(m)

# Save the map to an HTML file
m.save('clusters_map.html')

# Display the map
m